<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)

In [2]:
"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib

to_import = ['ABF_Quality_Control.ipynb',
          'Basic_Ephys.ipynb',
          'Firing_Rate_Gain.ipynb',
          'Simple_ABF_tools.ipynb',
          'analyze_IV.ipynb',
          'analyze_memtest.ipynb',
          'analyze_rheobase.ipynb',
          'fun_math.ipynb',
          'importing_abfs_from_dropbox.ipynb',
          'inputR_from_Iclamp.ipynb',
          'spike_latency.ipynb']

for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f


Cloning into 'EphysLib'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 85 (delta 42), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (85/85), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.5 MB/s 


In [39]:
'Import and catalog source data'
'RNF182 Dropbox'
data_source = "https://www.dropbox.com/sh/n9t8p257wnzlijk/AAC9Z36JodisyZjnM3mkJC3Xa?dl=0"
file_loc = get_drobox_folder(data_source, 'my_ephys_data.zip')
clear_output(wait=False)
file_naming_scheme = ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type']
abf_recordings_df,protocol_set = catalogue_recs(file_loc,file_naming_scheme)
print('Protocol_Names:')
_ = [print(p) for p in protocol_set]

Protocol_Names:
IC - Sag - D50pA
VC - Multi IV - 150ms
VC - MemTest-10ms-160ms
IC - R input
IC - Latentcy 800pA-1s
VC - 3min GapFree
IC - Rheobase
IC - Gain - D50pA


In [ ]:
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',
           'VC - 3min GapFree',
           'VC - Spontaneous-3min-(MT)']
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s']

abf_recordings_df, _ = purge_wrong_clamp(abf_recordings_df,VC_prot,IC_prot)

In [ ]:
def qc_sweep(abf, sampleRate, qc_args=True, to_plot=False):
    'Recieves a sweep and and calculates leak, noise and holding potential'
    'returns a dict of calculated values and a dict of boolean indicating'
    'pass/fail, (True/False)'


    if qc_args: qc_args = { 'max_leak': 100, 'HF_noise': 10,'HF_window': 0.015, 'LF_noise':10, 'IC_Vm_range':5, 'stim_buffer':250}

    abf = abf_or_name(abf)
    
    results={}

    buffer_mask = movmean((sweepC==sweepC[0])*1, qc_args['stim_buffer']/1000*sampleRate)==1
    no_stim_x = abf.sweepX[buffer_mask]
    no_stim_cmd = abf.sweepC[buffer_mask]
    no_stim_y = abf.sweepY[buffer_mask]

    baseline_y = np.mean( no_stim_y )
    baseline_cmd = np.mean( no_stim_cmd )

    if 'mV' in abf.sweepLabelY:
        results['Vhold'] = baseline_y
        results['Ihold'] = baseline_cmd
    else:
        results['Ihold'] = baseline_y
        results['Vhold'] = baseline_cmd


In [ ]:
def qc_sweep(sweepX,sweepC,sweepY,command_offset,is_IC,is_VC,sampleRate,
             max_leak=100, 
             max_high_freq_noise = 10,
             max_low_freq_noise = 10,
             Vhold_range = 5, to_plot=False):
    'Recieves a sweep and and calculates leak, noise and holding potential'
    'returns a dict of calculated values and a dict of boolean indicating'
    'pass/fail, (True/False)'

    


    stim_buffer_time = 250 #ms
    filtered_command = movmean((sweepC==sweepC[0])*1, stim_buffer_time/1000*sampleRate)
    ss_no_stim_bool = filtered_command==1
    ss_no_stim_idx = np.arange(len(ss_no_stim_bool))[ss_no_stim_bool]
    no_stim_sig = sweepY[ss_no_stim_bool]
    no_stim_t = sweepX[ss_no_stim_idx]
    baseline = np.mean( no_stim_sig )

    QC_checks = {}
    QC_values = {}
    if is_IC:
        QC_values['V_hold'] = baseline
        QC_values['I_leak'] = command_offset
    if is_VC:
        QC_values['V_hold'] = command_offset
        QC_values['I_leak'] = baseline
    
    QC_checks['V_hold'] = abs(QC_values['V_hold'] - -70)< Vhold_range
    QC_checks['I_leak'] = QC_values['I_leak']<max_leak


    
    HF_noise_idx = ss_no_stim_idx[:int(0.0015*sampleRate)]
    HF_noise_signal = sweepY[ HF_noise_idx ] 
    HF_noise = rms_noise(HF_noise_signal)     
    
    QC_checks['HF_noise'] = HF_noise<max_high_freq_noise
    QC_values['HF_noise'] = HF_noise
    
    LF_noise_idx = ss_no_stim_idx[-int(0.1*sampleRate):] 
    if int(0.1*sampleRate)>len(LF_noise_idx): LF_noise_idx = np.random.choice(LF_noise_idx, size=int(0.1*sampleRate))
    LF_noise_signal = sweepY[ LF_noise_idx ] 
    LF_noise = rms_noise(LF_noise_signal)
    QC_checks['LF_noise'] = LF_noise<max_low_freq_noise
    QC_values['LF_noise'] = LF_noise

    HF_noise_time = sweepX[HF_noise_idx]
    LF_noise_time = sweepX[LF_noise_idx]
    LF_noise_time = np.sort(np.array(list(set(LF_noise_time))))

    return QC_checks, QC_values, [HF_noise_time, LF_noise_time]

In [ ]:










def purge_wrong_clamp(abf_recordings_df,VC_prot,IC_prot,verbose=False, to_plot=False):
    VC_match_list, IC_match_list = search_wrong_clamp(abf_recordings_df,VC_prot,IC_prot)
    both_good = [VC_match_list[i] and IC_match_list[i] for i in range(len(IC_match_list))]
    abf_recordings_df_purge = abf_recordings_df[both_good]
    both_bad = [not(a) for a in both_good]
    
    bad_file_names = abf_recordings_df.index[both_bad]
    if verbose:
        print('Bad files:')
        _ = [print('     ',f) for f in bad_file_names]
    if to_plot:
        for f in bad_file_names:
            plot_sweeps_and_command(f)


    return abf_recordings_df_purge, bad_file_names

def search_wrong_clamp(abf_recordings_df,VC_prot,IC_prot):
    'takes a df with an index of filename and checks for correct clamping'
    VC_match_list = []
    IC_match_list = []
    for f in abf_recordings_df.index:
        is_VC = clamp_check(f)['is_VC']
        is_VC_prot = abf_recordings_df.loc[f,'protocol'] in VC_prot
        VC_match_list.append(is_VC == is_VC_prot)

        is_IC = clamp_check(f)['is_VC']
        is_IC_prot = abf_recordings_df.loc[f,'protocol'] in IC_prot
        IC_match_list.append(is_IC == is_VC_prot)
    return VC_match_list, IC_match_list



def clamp_check(abf):
    abf = abf_or_name(abf)
    if 'mV' in abf.sweepLabelY:
        is_IC = True
        is_VC = False
    if 'pA' in abf.sweepLabelY:
        is_IC = False
        is_VC = True
    return {'is_IC': is_IC, 'is_VC': is_VC}